Neural Network in Pytorch is constructed using "torch.nn", which depends on autograd to define models and differentiate them

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
## define a new network
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channel, 5x5 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # linear function: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2 , 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square, can specify only a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # flatten features
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

In [8]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Only have to define forward function, since backward is automatically provided by autograd framework.

the learnable parameters are returned by nn.Module.parameters()

In [7]:
params = list(net.parameters())
print(len(params))
# conv1's weight
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


Try this net on MNIST dataset, need to first resize images to 32 x 32

In [9]:
input = torch.rand(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0525, -0.0401, -0.0382, -0.1574,  0.0606,  0.0146,  0.0276, -0.0648,
          0.0372,  0.0452]], grad_fn=<AddmmBackward>)


In [10]:
net.zero_grad()
out.backward(torch.randn(1,10))

Torch.nn only support mini-batches. nn.Conv2d will take in 4D tensor of nSamples x nChannels x Height x Width.

In [12]:
output = net(input)
target = torch.randn(10) # dummy target
target = target.view(1, -1) # make same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4243, grad_fn=<MseLossBackward>)


In [14]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [15]:
net.zero_grad() # zero out the gradient buffer
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0007,  0.0009, -0.0076, -0.0013,  0.0041,  0.0036])


In [16]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [17]:
import torch.optim as optim


In [18]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

loss.backward()

# does the update
optimizer.step()